In [ ]:
import keras
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, MaxPool2D
from keras.layers.core import Lambda
from keras.preprocessing import image
from tensorflow.keras import datasets, layers, models, losses
from tensorflow.keras.models import Model, Sequential
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import *
from keras.models import load_model
from keras.layers.core import Lambda
import os
from keras.layers import *
from sklearn.model_selection import KFold, StratifiedKFold
from keras.backend import int_shape
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense, multiply, Permute, Concatenate, Add, Activation, Lambda, Average
from keras import backend as K
from keras.activations import sigmoid
from keras.backend import sigmoid
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
from tensorflow.python.ops.gen_math_ops import mul
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sn

In [ ]:
train_data = ImageDataGenerator(rescale=1./255, rotation_range = 10,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   horizontal_flip = True )
train_data = train_data.flow_from_directory('gluacoma_dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')
test_data = ImageDataGenerator(rescale=1./255)
test_data = test_data.flow_from_directory('gluacoma_dataset/test',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            shuffle = False,
                                            class_mode = 'categorical')

In [ ]:
def swish(x, beta = 1):
    return(x * sigmoid(beta * x))
get_custom_objects().update({'swish': Activation(swish)})

In [ ]:
def channel_shuffle(x, groups):  
    _, width, height, channels = x.get_shape().as_list()
    group_ch = channels // groups
 
    x = Reshape([width, height, group_ch, groups])(x)
    x = Permute([1, 2, 4, 3])(x)
    x = Reshape([width, height, channels])(x)
    return x

In [ ]:
def conv_module(x,k):
    a=Conv2D(k,(1,1),padding='same')(x)
    a=BatchNormalization()(a)
    a=Activation('swish')(a)

    y=Conv2D(k//2,(3,3),padding='same')(x)
    y=BatchNormalization()(y)
    y=Activation('swish')(y)
    y=Conv2D(k//2,(3,3),padding='same')(y)
    y=BatchNormalization()(y)
    y=Activation('swish')(y)

    z=Conv2D(k,(3,3),padding='same')(x)
    z=BatchNormalization()(z)
    z=Activation('swish')(z)
    c=tf.keras.layers.Concatenate()([y,z])

    d=DepthwiseConv2D(3,padding='same')(c)
    d=BatchNormalization()(d)
    d=Activation('swish')(d)
    x=tf.keras.layers.Concatenate()([d,c])
    x=tf.keras.layers.Concatenate()([x,a])
    x=Conv2D(k,(1,1),padding='same')(x)
    x=BatchNormalization()(x)
    x=Activation('swish')(x)

    return x

In [ ]:
def CSDA(inputs):
    shape=K.int_shape(inputs)
    x=MaxPooling2D(pool_size=(shape[1],shape[2])) (inputs)
    x=Conv2D(shape[3]//4,1, padding='same') (x)
    x=Activation('swish') (x)
    x=Conv2D(shape[3],1, padding='same') (x)
    x=Activation('sigmoid') (x)
    y=AveragePooling2D(pool_size=(shape[1],shape[2])) (inputs)
    y=Conv2D(shape[3]//4,1, padding='same') (y)
    y=Activation('swish') (x)
    y=Conv2D(shape[3],1, padding='same') (y)
    y=Activation('sigmoid') (y)
    z=tf.keras.layers.Average()([x,y])
    C_A=tf.math.multiply(z,inputs)
    
    a=Lambda(lambda x: K.max(x,axis=3,keepdims=True))  (inputs)
    a=Conv2D(shape[3] // shape[3],1, padding='same') (a)
    a=Activation('sigmoid') (a)
    b=Lambda(lambda x: K.mean(x,axis=3,keepdims=True))  (inputs)
    b=Conv2D(shape[3] // shape[3],1, padding='same') (a)
    b=Activation('sigmoid') (a)
    c=tf.keras.layers.Average()([a,b])
    S_A=tf.math.multiply(c,inputs)
    v=tf.keras.layers.Add()([C_A,S_A])
    d=channel_shuffle(inputs, 8)
    x=tf.keras.layers.Add()([v,d])
    return x

In [ ]:
def get_base_model():
  input_shape = (224,224,3)
  inputs = Input(shape=(224,224,3))
  x=Conv2D(32,3, padding='same') (inputs)
  x=BatchNormalization()(x)
  x=Activation('swish')(x)
  x=MaxPooling2D()(x)
  x=conv_module(x,64)
  x=MaxPooling2D()(x)
  x=CSDA(x)
  x=conv_module(x,128)
  x=MaxPooling2D()(x)
  x=CSDA(x)
  x=conv_module(x,256)
  x=MaxPooling2D()(x)
  x=CSDA(x)
  x=Conv2D(256,3, padding='same') (x)
  x=BatchNormalization()(x)
  x=Activation('swish')(x)
  x=MaxPooling2D()(x)
  x=CSDA(x)
  x=GlobalAveragePooling2D()(x)
  x=Dense(2,activation='softmax')(x)
  model=Model(inputs, x)
  return model

In [ ]:
model=get_base_model()  
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics="acc")
model.summary()

In [ ]:
reduce_lr=ReduceLROnPlateau(monitor='loss', factor=0.4,patience=5, min_lr=0.00001)
filepath = "CDAM_Net.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [reduce_lr, checkpoint]
history = model.fit(train_data,epochs=1,batch_size=16,callbacks=callbacks_list)
results = model.evaluate(test_data, batch_size=16)
Y_pred = model.predict(test_data, 428 // 17)
print(test_data.classes)  
print(Y_pred)
y_pred = np.argmax(Y_pred, axis=1)
true_classes = test_data.classes
class_labels = list(test_data.class_indices.keys())
print('Confusion Matrix')
matrix = confusion_matrix(test_data.classes, y_pred)
plt.figure(figsize = (10,7))
names = ["Glaucoma", "Normal"]
group_counts = ["{0:0.0f}".format(value) for value in
                matrix.flatten()]
labels = [f"{v1}" for v1 in group_counts]
labels = np.asarray(labels).reshape(2,2)
sn.set(font_scale=2)
sn.heatmap(matrix, annot=labels, fmt='', xticklabels= names, yticklabels= names, annot_kws={"size": 19}, cmap='Blues')
plt.yticks(rotation=90)
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title('Confusion Matrix ')
plt.savefig('confusioncdam.png',dpi=1080)
plt.show()
print('Classification Report')
target_names = ['Glaucoma', 'Normal']
print(classification_report(test_data.classes, y_pred, target_names=target_names))